In [1]:
%load_ext autoreload
%autoreload 2
from agent import Agent
import json

# Load agent definition from the JSON file
import pathlib

agent_def_path = pathlib.Path("agent_definition.json")
if not agent_def_path.exists():
    raise FileNotFoundError(f"Agent definition file not found: {agent_def_path.resolve()}")

with agent_def_path.open("r", encoding="utf-8") as f:
    agent_definition = json.load(f)

# # Show the loaded definition

agent_definition.keys()


dict_keys(['azure_agent', 'Ollama_agent'])

In [2]:
agent_name = "Ollama_agent"
print(f"Creating agent: {agent_name}")
print(f"Agent config: {agent_definition[agent_name]}")
ff_agent = await Agent.create(agent_definition[agent_name])
print("Available tools: ", [tool.name for tool in ff_agent.available_tools.tools])
result = ff_agent.run_agent("What teams are in the league?")

Creating agent: Ollama_agent
Agent config: {'deployment_name': 'gpt-oss:20b', 'endpoint': 'http://ollama.home', 'system_message': 'you are an expert and help find answers. you have tools, and can use them', 'servers': {'ff_tools': {'url': 'http://192.168.86.103:8000/mcp', 'type': 'http'}, 'sql_tools': {'url': 'https://somesseserver.azurewebsites.us/sse', 'type': 'http'}}, 'inputs': []}


ERROR:root:Failed to connect to MCP server sql_tools: 'NoneType' object has no attribute 'list_tools'


Available tools:  ['get_teams', 'get_roster', 'get_player_data', 'get_waivers', 'update_roster', 'make_waiver_claim', 'analyze_trade_values', 'get_player_rank']


In [3]:
# agent_name = "azure_agent"
# print(f"Creating agent: {agent_name}")
# print(f"Agent config: {agent_definition[agent_name]}")
# ff_agent = await Agent.create(agent_definition[agent_name])
# result = await ff_agent.run_agent("Who is active on Blaine's team?")

In [4]:

agents = list(agent_definition.keys())
agents

['azure_agent', 'Ollama_agent']

In [7]:
async for res in result:
    print(res)

INFO:semantic_kernel.connectors.ai.chat_completion_client_base:processing 1 tool calls in parallel.
INFO:semantic_kernel.kernel:Calling ff_tools-get_teams function with args: {}
INFO:semantic_kernel.functions.kernel_function:Function ff_tools-get_teams invoking.
INFO:semantic_kernel.kernel:Calling ff_tools-get_teams function with args: {}
INFO:semantic_kernel.functions.kernel_function:Function ff_tools-get_teams invoking.
INFO:semantic_kernel.functions.kernel_function:Function ff_tools-get_teams succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.039998s
INFO:semantic_kernel.functions.kernel_function:Function ff_tools-get_teams succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.039998s


{'thoughts': 'User asks: "What teams are in the league?" Likely need to call ff_tools_get_teams.We need to answer "What teams are in the league?" We have the output from get_teams. That gives a dictionary mapping team names to details. We should format the answer: list the team names. Possibly give short names. The user asked "What teams are in the league?" So we can say:\n\n- America\'s Fantasy Team\n- Chance (not Chase)\n- Gettin\' Breecey Wit It\n- GRI... etc.\n\nBetter to list all.\n\nLet\'s parse the JSON. It lists keys: America\'s Fantasy Team, Chance (not Chase), Gettin\' Breecey Wit It, GRI..., In GPT We Trust, Joe Buck Yourself, Kool name still pending, Mr. Rodgers Naber-hood, Off To Grandma, Penix Pumpers, Redstone Red Rockets, Yinzer Fan Club.\n\nThus 12 teams.\n\nWe can output that list. Also maybe provide short names. But user just asked for teams. So list names.\n\nThus answer.', 'tool_calls': [{'ff_tools-get_teams': {}}], 'messages': 'Here are all the teams currently in 

In [5]:
result.get("thoughts")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-57' coro=<MCPPluginBase._inner_connect() done, defined at c:\Users\blain\Documents\git\semantic_kernel_mcp\.venv\Lib\site-packages\semantic_kernel\connectors\mcp.py:264> exception=KernelPluginInvalidConfigurationError('Failed to connect to the MCP server. Please check your configuration.')>
  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\blain\Documents\git\semantic_kernel_mcp\.venv\Lib\site-packages\semantic_kernel\connectors\mcp.py", line 267, in _inner_connect
  |     transport = await self._exit_stack.enter_async_context(self.get_mcp_client())
  |                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\blain\AppData\Roaming\uv\python\cpython-3.12.10-windows-x86_64-none\Lib\contextlib.py", line 659, in enter_async_context
  |     result = await _enter(cm)
  |              ^^^^^^^^^^^^^^^^
  |   File "C:\Users\blain\AppData\Roam

AttributeError: 'async_generator' object has no attribute 'get'

In [ ]:
result.get("tool_calls")[0]

In [ ]:
result.get("messages")

In [ ]:
# await ff_agent.run_agent("what is in my db??")